In [1]:
%pylab inline

import pandas as pd
from datetime import datetime

Populating the interactive namespace from numpy and matplotlib


# 1. 读取数据

In [3]:
ls ../../official_data

config1.csv*             item_feature2.csv*       sample_submission.csv
config2.csv*             item_store_feature1.csv*
item_feature1.csv*       item_store_feature2.csv*


In [4]:
# 时间处理函数
date_func = lambda x : datetime.strptime(str(x), "%Y%m%d")

In [5]:
path_prefix = "../../official_data/"
item_global_record_filepath = path_prefix + "item_feature2.csv"
item_local_record_filepath = path_prefix + "item_store_feature2.csv"
config_filepath = path_prefix + "config2.csv"

## 1.1 全国数据

In [6]:
item_global_feature_header =["date","item_id","cate_id","cate_level_id","brand_id","supplier_id","pv_ipv","pv_uv","cart_ipv","cart_uv","collect_uv",
         "num_gmv","amt_gmv","qty_gmv","unum_gmv","amt_alipay","num_alipay","qty_alipay","unum_alipay","ztc_pv_ipv",
         "tbk_pv_ipv","ss_pv_ipv","jhs_pv_ipv","ztc_pv_uv","tbk_pv_uv","ss_pv_uv","jhs_pv_uv","num_alipay_njhs",
         "amt_alipay_njhs","qty_alipay_njhs","unum_alipay_njhs"]

In [11]:
item_global_records = pd.read_csv(item_global_record_filepath, header = None, names = item_global_feature_header)
item_global_records.sort_index(by = ["date"], inplace = True)
item_global_records["store_code"] = "0"
item_global_records.date = item_global_records.date.map(date_func)

## 1.2 分区数据

In [8]:
item_local_feature_header = ["date","item_id","store_code","cate_id","cate_level_id","brand_id","supplier_id",
    "pv_ipv","pv_uv","cart_ipv","cart_uv","collect_uv","num_gmv","amt_gmv","qty_gmv","unum_gmv","amt_alipay",
    "num_alipay","qty_alipay","unum_alipay","ztc_pv_ipv","tbk_pv_ipv","ss_pv_ipv","jhs_pv_ipv","ztc_pv_uv",
    "tbk_pv_uv","ss_pv_uv","jhs_pv_uv","num_alipay_njhs","amt_alipay_njhs","qty_alipay_njhs","unum_alipay_njhs"]

In [12]:
item_local_records = pd.read_csv(item_local_record_filepath, header = None, names = item_local_feature_header)
item_local_records.sort_index(by = ["date"], inplace = True)
item_local_records.date = item_local_records.date.map(date_func)

# 1.3 合并全国和地区数据

In [13]:
records = pd.concat([item_global_records, item_local_records])
records.sort_index(by = ["date", "store_code"], inplace = True)
records.head()

,amt_alipay,amt_alipay_njhs,amt_gmv,brand_id,cart_ipv,cart_uv,cate_id,cate_level_id,collect_uv,date,...,ss_pv_uv,store_code,supplier_id,tbk_pv_ipv,tbk_pv_uv,unum_alipay,unum_alipay_njhs,unum_gmv,ztc_pv_ipv,ztc_pv_uv
601213,47.21,47.21,47.2,41,1,1,22,4,0,2014-10-10,...,1,1,662,0,0,1,1,1,0,0
110995,0.00,0.00,0.0,40,0,0,39,12,0,2014-10-10,...,0,1,430,0,0,0,0,0,0,0
742039,0.00,0.00,0.0,406,0,0,39,12,0,2014-10-10,...,0,1,1335,0,0,0,0,0,0,0
736714,0.00,0.00,0.0,687,0,0,17,12,0,2014-10-10,...,1,1,1986,0,0,0,0,0,0,0
115031,0.00,0.00,0.0,431,0,0,26,8,0,2014-10-10,...,0,1,80,0,0,0,0,0,0,0


## 1.3 补偿数据

In [14]:
config_header = ["item_id", "store_code", "a_b"]

In [15]:
item_configs = pd.read_csv(config_filepath, header = None, names = config_header)
func_a = lambda x : x.strip().split("_")[0]
func_b = lambda x : x.strip().split("_")[1]
item_configs["a"] = item_configs["a_b"].map(func_a).astype(float)
item_configs["b"] = item_configs["a_b"].map(func_b).astype(float)
item_configs.drop("a_b", axis = 1, inplace = True)
item_configs.head()

,item_id,store_code,a,b
0,58452,all,46.38,11.86
1,36132,all,45.79,85.94
2,34092,all,43.65,83.86
3,62772,all,44.68,16.89
4,9597,all,41.04,17.70


# 2. 数据预处理

In [30]:
from datetime import date, datetime, timedelta
def fill(df):
    start_date = min(df.date)
    end_date = pd.to_datetime(date(2016,1,11))
    len_date = (end_date - start_date).days
    date_list = [start_date + timedelta(days = d) for d in range(len_date)]
    date_list = date_list * 6
    store_code_list = []
    store_code_list.extend(["0"] * len_date)
    store_code_list.extend(["1"] * len_date)
    store_code_list.extend(["2"] * len_date)
    store_code_list.extend(["3"] * len_date)
    store_code_list.extend(["4"] * len_date)
    store_code_list.extend(["5"] * len_date)
    record_dict = {"date": date_list, "store_code": store_code_list}
    new_df = pd.DataFrame(record_dict)
    new_df = pd.merge(new_df, df, on = ["date", "store_code"], how = "left")
    fill_value = {"item_id":df.item_id.median(),"cate_id":df.cate_id.median(),\
                  "cate_level_id":df.cate_level_id.median(),"brand_id":df.brand_id.median(),\
                  "supplier_id":df.supplier_id.median()}
    new_df.fillna(fill_value, inplace = True)
    new_df.fillna(0, inplace = True)
    return new_df

In [34]:
records_filled = records.copy().groupby(["item_id"], group_keys = False).apply(fill)
records_filled.sort_index(by = ["item_id", "store_code", "date"], inplace = True)
records_filled.head()

,date,store_code,item_id,amt_alipay,amt_alipay_njhs,amt_gmv,brand_id,cart_ipv,cart_uv,cate_id,...,overall_njhs_median,current_month_njhs_mean,njhs_pre_1,njhs_pre_2,njhs_pre_3,njhs_pre_4,njhs_pre_5,njhs_pre_6,njhs_pre_7,qty_alipay_jhs
0,2015-11-28,0,33,885.3,885.3,4578.87,390,0,0,39,...,0,0.111111,1,1,1,1,1,1,1,0
1,2015-11-29,0,33,0.0,0.0,0.00,390,1,1,39,...,0,0.111111,1,1,1,1,1,1,1,0
2,2015-11-30,0,33,885.3,885.3,4578.87,390,3,3,39,...,0,0.111111,0,1,1,1,1,1,1,0
3,2015-12-01,0,33,0.0,0.0,0.00,390,0,0,39,...,0,0.053763,1,0,1,1,1,1,1,0
4,2015-12-02,0,33,0.0,0.0,0.00,390,0,0,39,...,0,0.053763,0,1,0,1,1,1,1,0


# 3. 特征提取

## 3.1 时间特征

In [35]:
dt = pd.DatetimeIndex(records_filled["date"])

In [36]:
records_filled["date"] = dt.date
records_filled["year"] = dt.year
records_filled["month"] = dt.month
records_filled["day"] = dt.day
records_filled["dow"] = dt.dayofweek
records_filled["woy"] = dt.weekofyear

In [37]:
records_filled.head()

,date,store_code,item_id,amt_alipay,amt_alipay_njhs,amt_gmv,brand_id,cart_ipv,cart_uv,cate_id,...,overall_njhs_median,current_month_njhs_mean,njhs_pre_1,njhs_pre_2,njhs_pre_3,njhs_pre_4,njhs_pre_5,njhs_pre_6,njhs_pre_7,qty_alipay_jhs
0,2015-11-28,0,33,885.3,885.3,4578.87,390,0,0,39,...,0,0.111111,1,1,1,1,1,1,1,0
1,2015-11-29,0,33,0.0,0.0,0.00,390,1,1,39,...,0,0.111111,1,1,1,1,1,1,1,0
2,2015-11-30,0,33,885.3,885.3,4578.87,390,3,3,39,...,0,0.111111,0,1,1,1,1,1,1,0
3,2015-12-01,0,33,0.0,0.0,0.00,390,0,0,39,...,0,0.053763,1,0,1,1,1,1,1,0
4,2015-12-02,0,33,0.0,0.0,0.00,390,0,0,39,...,0,0.053763,0,1,0,1,1,1,1,0


## 3.2 全局特征

In [38]:
overall_njhs_mean = records_filled.groupby("item_id").qty_alipay_njhs.mean()
overall_njhs_min = records_filled.groupby("item_id").qty_alipay_njhs.min()
overall_njhs_max = records_filled.groupby("item_id").qty_alipay_njhs.max()
overall_njhs_median = records_filled.groupby("item_id").qty_alipay_njhs.median()

records_filled["overall_njhs_mean"] = records_filled["item_id"].map(overall_njhs_mean.to_dict())
records_filled["overall_njhs_min"] = records_filled["item_id"].map(overall_njhs_min.to_dict())
records_filled["overall_njhs_max"] = records_filled["item_id"].map(overall_njhs_max.to_dict())
records_filled["overall_njhs_median"] = records_filled["item_id"].map(overall_njhs_median.to_dict())

## 3.3 局部特征

In [39]:
month_feature = records_filled.groupby(["item_id", "month"]).qty_alipay_njhs.mean()
month_feature = month_feature.to_dict()
records_filled["current_month_njhs_mean"] = records_filled.apply(lambda x : month_feature[(x["item_id"], x["month"])], axis = 1)

## 3.4 历史特征

In [40]:
def shiftFunc(df, window_size, cols):
    item_series_len = df.shape[0]
    window_size = min(window_size + 1, item_series_len)
    for w in range(1, window_size):
        item_series = [df.qty_alipay_njhs[0]] * w
        item_series.extend(df.qty_alipay_njhs[0:-w])
        df[cols[w-1]] = item_series
    return df

In [41]:
records_filled.set_index(["item_id", "date"], inplace = True)
shift_cols = ["njhs_pre_1", "njhs_pre_2", "njhs_pre_3", "njhs_pre_4", "njhs_pre_5", "njhs_pre_6", "njhs_pre_7"]
records_filled_shift = records_filled.groupby(level = ["item_id"]).apply(lambda x : shiftFunc(x, 7, shift_cols))
records_filled_shift.reset_index(inplace = True)

In [42]:
records_filled_shift["qty_alipay_jhs"] = records_filled_shift["qty_alipay"] - records_filled_shift["qty_alipay_njhs"]

In [73]:
records_final = records_filled_shift
records_final.fillna(0, inplace = True)
records_final.sort_index(by = ["date", "item_id", "store_code"], inplace = True)
records_filled.head()

store_code  amt_alipay  amt_alipay_njhs  amt_gmv  brand_id  \
item_id date                                                                    
33      2015-11-28          0       885.3            885.3  4578.87       390   
        2015-11-29          0         0.0              0.0     0.00       390   
        2015-11-30          0       885.3            885.3  4578.87       390   
        2015-12-01          0         0.0              0.0     0.00       390   
        2015-12-02          0         0.0              0.0     0.00       390   

                    cart_ipv  cart_uv  cate_id  cate_level_id  collect_uv  \
item_id date                                                                
33      2015-11-28         0        0       39             12           0   
        2015-11-29         1        1       39             12           0   
        2015-11-30         3        3       39             12           1   
        2015-12-01         0        0       39             12           1   
        2015-12-02         0        0       39             12           0   

                         ...        overall_njhs_median  \
item_id date             ...                              
33      2015-11-28       ...                          0   
        2015-11-29       ...                          0   
        2015-11-30       ...                          0   
        2015-12-01       ...                          0   
        2015-12-02       ...                          0   

                    current_month_njhs_mean  njhs_pre_1  njhs_pre_2  \
item_id date                                                          
33      2015-11-28                 0.111111           1           1   
        2015-11-29                 0.111111           1           1   
        2015-11-30                 0.111111           0           1   
        2015-12-01                 0.053763           1           0   
        2015-12-02                 0.053763           0           1   

                    njhs_pre_3  njhs_pre_4  njhs_pre_5  njhs_pre_6  \
item_id date                                                         
33      2015-11-28           1           1           1           1   
        2015-11-29           1           1           1           1   
        2015-11-30           1           1           1           1   
        2015-12-01           1           1           1           1   
        2015-12-02           0           1           1           1   

                    njhs_pre_7  qty_alipay_jhs  
item_id date                                    
33      2015-11-28           1               0  
        2015-11-29           1               0  
        2015-11-30           1               0  
        2015-12-01           1               0  
        2015-12-02           1               0  

[5 rows x 48 columns]

# 4. 数据集划分

In [53]:
feature_cols = ["item_id", "store_code", "year", "month", "day", "dow", "woy",\
                 "cate_level_id", "cate_id", "brand_id", "supplier_id",
                 "overall_njhs_mean", "overall_njhs_min", "overall_njhs_max", "overall_njhs_median",\
                 "current_month_njhs_mean",\
                 "njhs_pre_1", "njhs_pre_2", "njhs_pre_3", "njhs_pre_4", "njhs_pre_5", "njhs_pre_6"]
target_cols = ["qty_alipay_njhs"]

In [65]:
X_all = records_final[feature_cols]
X_train = records_final[records_final.date <= date(2015,12,13)][feature_cols]
y_train = records_final[records_final.date <= date(2015,12,13)][target_cols]
X_val = records_final[(records_final.date >= date(2015,12,14)) & (records_final.date <= date(2015,12,27))][feature_cols]
y_val = records_final[(records_final.date >= date(2015,12,14)) & (records_final.date <= date(2015,12,27))][target_cols]
X_test = records_final[records_final.date >= date(2015,12,28)][feature_cols]

X_all = np.array(X_all)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)

X_all = X_all.astype(int)
X_train = X_train.astype(int)
X_val = X_val.astype(int)
X_test = X_test.astype(int)

In [66]:
print "training size: " + str(len(X_train))
print "validation size: " + str(len(X_val))
print "test size: " + str(len(X_test))
print "feature size: " + str(len(X_train[0]))

training size: 1307124
validation size: 80844
test size: 80892
feature size: 22


# 5.序列化特征文件

In [68]:
import pickle
from sklearn import preprocessing

les = []
category_feature_len = 11
for col in range(category_feature_len):
    le = preprocessing.LabelEncoder()
    le.fit(X_all[:, col])
    les.append(le)
    X_train[:, col] = le.transform(X_train[:, col])
    X_val[:, col] = le.transform(X_val[:, col])
    X_test[:, col] = le.transform(X_test[:, col])
X_train = X_train.astype(int)
X_val = X_val.astype(int)
X_test = X_test.astype(int)
    
with open("../data/les.pickle", "wb") as f:
    pickle.dump(les, f, -1)

with open("../data/train_xy.pickle", "wb") as f:
    pickle.dump((X_train, y_train), f, -1)

with open("../data/val_xy.pickle", "wb") as f:
    pickle.dump((X_val, y_val), f, -1)    
    
with open("../data/test_x.pickle", "wb") as f:
    pickle.dump(X_test, f, -1) 